In [1]:
import pandas as pd
import numpy as np
# Cargamos datos
df_2004 = pd.read_stata("C:/Users/Fran/Downloads/usu_individual_T104.dta")
df_2024 = pd.read_excel("C:/Users/Fran/Downloads/EPH_usu_1_Trim_2024_xls/EPH_usu_1er_Trim_2024_xlsx/usu_individual_T124.xlsx")

# Filtramos por Neuquén-Plottier (AGLOMERADO = 17)
df_2004_neuquen = df_2004[df_2004['aglomerado'] == 17].copy() 
df_2024_neuquen = df_2024[df_2024['AGLOMERADO'] == 17].copy()

# Estandarizamos nombres de columnas
df_2024_neuquen.columns = df_2024_neuquen.columns.str.lower()

# Agregamos columna de año antes de concatenar
df_2004_neuquen['anio'] = 2004
df_2024_neuquen['anio'] = 2024

# Unificamos bases manteniendo solo columnas comunes
common_cols = list(set(df_2004_neuquen.columns) & set(df_2024_neuquen.columns))
df_unificado = pd.concat([
    df_2004_neuquen[common_cols],
    df_2024_neuquen[common_cols]
], ignore_index=True)


# Excel con configuración para mejor visualización
!pip install xlsxwriter
with pd.ExcelWriter("EPH_Neuquen_Plottier_2004_2024.xlsx", 
                   engine='xlsxwriter',
                   engine_kwargs={'options': {'strings_to_numbers': True}}) as writer:
    df_unificado.to_excel(writer, index=False)
    
    

print("\nResultados:")
print(f"- Registros 2004: {len(df_2004_neuquen)}")
print(f"- Registros 2024: {len(df_2024_neuquen)}")
print(f"- Total unificado: {len(df_unificado)}")
print("\nArchivos guardados:")
print("- EPH_Neuquen_Plottier_2004_2024.xlsx (para visualización)")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Fran/Downloads/usu_individual_T104.dta'

In [ ]:
#Ponemos todos los nombres de las columnas en maypusculas para evitar errores futuros
df_2004_neuquen.columns = df_2004_neuquen.columns.str.upper()
df_2024_neuquen.columns = df_2024_neuquen.columns.str.upper()

# Seleccionar 15 variables clave
variables_interes = [
    'CH04', 'CH06', 'ESTADO', 'NIVEL_ED', 'IPCF', 'PP07H', 
    'PP07I', 'V2_M', 'V5_M', 'V8_M', 'V12_M', 'P21', 'CAT_INAC', 'CH07', 'CH08'
]

# Verificar NA por año
na_2004 = df_2004_neuquen[variables_interes].isna().sum().rename("NA_2004")
na_2024 = df_2024_neuquen[variables_interes].isna().sum().rename("NA_2024")
tabla_na = pd.concat([na_2004, na_2024], axis=1)

# Mostrar tabla
print("VALORES FALTANTES POR AÑO:")
display(tabla_na)


In [ ]:
import pandas as pd
import numpy as np

def encontrar_columna(base_name, df_columns):
    
    posibles_nombres = [
        base_name,
        base_name.lower(),
        base_name.upper(),
        base_name.title(),
       ]
    
    for nombre in posibles_nombres:
        if nombre in df_columns:
            return nombre
    return None

def limpiar_datos_eph(df, verbose=True):

    reglas_limpieza = {
        'CH04': {'tipo': 'categorica', 'valores_na': [9], 'valores_validos': [1, 2]},
        'CH06': {'tipo': 'numerica', 'rango_valido': (0, 130)},
        'ESTADO': {'tipo': 'categorica', 'valores_na': [0, 9], 'valores_validos': [1, 2, 3]},
        'NIVEL_ED': {'tipo': 'categorica', 'valores_na': [0, 9]},
        'IPCF': {'tipo': 'numerica', 'valores_na': [-9, -8, -7]},
        'P21': {'tipo': 'numerica', 'valores_na': [-9, -8, -7]},
        'PPO7H': {'tipo': 'numerica', 'rango_valido': (0, 168)},
        'PPO7I': {'tipo': 'numerica', 'rango_valido': (0, 168)},
        'V2_M': {'tipo': 'categorica', 'valores_na': [0, 9]},
        'V5_M': {'tipo': 'categorica', 'valores_na': [0, 9]},
        'V8_M': {'tipo': 'categorica', 'valores_na': [0, 9]},
        'V12_M': {'tipo': 'categorica', 'valores_na': [0, 9]},
        'CH07': {'tipo': 'categorica', 'valores_na': [0, 9]},
        'CH08': {'tipo': 'categorica', 'valores_na': [0, 9]},
        'CAT_INAC': {'tipo': 'categorica', 'valores_na': [0, 9]}
    }
    
    for var_base, reglas in reglas_limpieza.items():
        nombre_real = encontrar_columna(var_base, df.columns)
        
        if nombre_real is None:
            if verbose:
                print(f"No se encontró {var_base} (buscada como: {var_base}, {var_base.lower()}, {var_base.upper()})")
            continue
            
        try:
            # Limpieza de valores NA
            if 'valores_na' in reglas:
                df[nombre_real] = df[nombre_real].replace({v: np.nan for v in reglas['valores_na']})
            
            # Conversión de tipo
            if reglas['tipo'] == 'numerica':
                df[nombre_real] = pd.to_numeric(df[nombre_real], errors='coerce')
                if 'rango_valido' in reglas:
                    df[nombre_real] = df[nombre_real].clip(*reglas['rango_valido'])
            
            elif reglas['tipo'] == 'categorica':
                if 'valores_validos' in reglas:
                    df[nombre_real] = df[nombre_real].where(df[nombre_real].isin(reglas['valores_validos']), np.nan)
                df[nombre_real] = df[nombre_real].astype('category')
                
        except Exception as e:
            if verbose:
                print(f"Error al limpiar {nombre_real}: {str(e)}")
    
    return df

def guardar_excel_limpio(df, excel_limpio_tp2):
    
    try:
        with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
            df.to_excel(writer, index=False, sheet_name='Datos_Limpios')
            
                
        print(f"\nArchivo guardado exitosamente: {nombre_excel}")
    except Exception as e:
        print(f"\nError al guardar el archivo Excel: {str(e)}")

#Limpiar los datos
df_2004_clean = limpiar_datos_eph(df_2004)

df_2024_clean = limpiar_datos_eph(df_2024)

# Unificar los datos limpios
df_completo_clean = pd.concat([df_2004_clean, df_2024_clean], keys=['2004', '2024'])

# Guardar en Excel 
nombre_excel = "EPH_Neuquen_Plottier_Limpio.xlsx"
nombre_archivo = "EPH_Neuquen_Plottier_Limpio.xlsx"

guardar_excel_limpio(df_completo_clean, nombre_excel)

# Resultado final
print("\nProceso completado exitosamente!")
print(f"Archivo Excel generado: {nombre_excel}")
print(f"Total registros 2004: {len(df_2004_clean)}")
print(f"Total registros 2024: {len(df_2024_clean)}")
print(f"Total registros unificados: {len(df_completo_clean)}")


In [ ]:
import matplotlib.pyplot as plt

# Configuración inicial
plt.style.use('seaborn')
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Datos 2004
sexo_2004 = df_2004_clean['ch04'].value_counts(normalize=True).sort_index() * 100
sexo_2004.plot(kind='bar', ax=axes[0], color=['skyblue', 'salmon'])
axes[0].set_title('Distribución por Sexo - 2004')
axes[0].set_xlabel('Sexo')
axes[0].set_ylabel('Porcentaje')
axes[0].set_xticklabels(['Varón', 'Mujer'], rotation=0)
axes[0].set_ylim(0, 60)

# Datos 2024
sexo_2024 = df_2024_clean['ch04'].value_counts(normalize=True).sort_index() * 100
sexo_2024.plot(kind='bar', ax=axes[1], color=['skyblue', 'salmon'])
axes[1].set_title('Distribución por Sexo - 2024')
axes[1].set_xlabel('Sexo')
axes[1].set_xticklabels(['Varón', 'Mujer'], rotation=0)
axes[1].set_ylim(0, 60)

plt.tight_layout()
plt.savefig('composicion_sexo.png', dpi=300)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Selección de variables para correlación (adaptar según nombres reales)
vars_corr = ['ch04', 'ch06', 'ch07', 'ch08', 'nivel_ed', 'estado', 'cat_inac', 'ipcf']
vars_disponibles = [v for v in vars_corr if v in df_2004_clean.columns]

# Matriz para 2004
plt.figure(figsize=(12, 8))
corr_2004 = df_2004_clean[vars_disponibles].corr(numeric_only=True)
sns.heatmap(corr_2004, annot=True, fmt=".2f", cmap='coolwarm', 
            center=0, vmin=-1, vmax=1,
            square=True, linewidths=.5, cbar_kws={"shrink": .8})
plt.title('Matriz de Correlación 2004 - Neuquén/Plottier', pad=20)
plt.tight_layout()
plt.savefig('correlacion_2004_seaborn.png', dpi=300, bbox_inches='tight')
plt.show()

# Matriz para 2024
plt.figure(figsize=(12, 8))
corr_2024 = df_2024_clean[vars_disponibles].corr(numeric_only=True)
sns.heatmap(corr_2024, annot=True, fmt=".2f", cmap='coolwarm',
            center=0, vmin=-1, vmax=1,
            square=True, linewidths=.5, cbar_kws={"shrink": .8})
plt.title('Matriz de Correlación 2024 - Neuquén/Plottier', pad=20)
plt.tight_layout()
plt.savefig('correlacion_2024_seaborn.png', dpi=300, bbox_inches='tight')
plt.show()

In [13]:
# Parte III: Conociendo a los ocupados y desocupados (Neuquén)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar bases
base_2004 = pd.read_stata("usu_individual_T104.dta")
base_2024 = pd.read_excel("usu_individual_T124.xls")

# Filtrar por la región de Neuquén (REGION = 42)
base_2004 = base_2004[base_2004["REGION"] == 42]
base_2024 = base_2024[base_2024["REGION"] == 42]

# Agregar columna de año
base_2004["anio"] = 2004
base_2024["anio"] = 2024

# Unir bases
base_total = pd.concat([base_2004, base_2024], ignore_index=True)

# ¿Cuántos desocupados, inactivos y promedio IPCF por ESTADO?
ocupacion_stats = base_total.groupby(["anio", "ESTADO"]).agg(
    cantidad=("ESTADO", "count"),
    ipc_promedio=("IPCF", "mean")
).reset_index()
print("--- Estadísticas por estado de actividad ---")
print(ocupacion_stats)

# Separar los que respondieron ESTADO y los que no
norespondieron = base_total[base_total["ESTADO"] == 0]
respondieron = base_total[base_total["ESTADO"] != 0]
print("\nCantidad que no respondieron ESTADO:", len(norespondieron))
print("Cantidad que respondieron ESTADO:", len(respondieron))

# PEA: 1 si ocupado o desocupado
respondieron["PEA"] = respondieron["ESTADO"].apply(lambda x: 1 if x in [1, 2] else 0)
pea_counts = respondieron.groupby(["anio", "PEA"]).size().unstack().rename(columns={0: "No PEA", 1: "PEA"})
pea_counts.plot(kind="bar", stacked=True)
plt.title("Composición por PEA (2004 vs 2024)")
plt.ylabel("Cantidad de personas")
plt.xlabel("Año")
plt.show()

# PET: 15 a 65 años
respondieron["PET"] = respondieron["CH06"].apply(lambda x: 1 if 15 <= x <= 65 else 0)
pet_counts = respondieron.groupby(["anio", "PET"]).size().unstack().rename(columns={0: "Fuera PET", 1: "PET"})
pet_counts.plot(kind="bar", stacked=True)
plt.title("Composición por PET (2004 vs 2024)")
plt.ylabel("Cantidad de personas")
plt.xlabel("Año")
plt.show()

# Variable desocupado
respondieron["desocupado"] = respondieron["ESTADO"].apply(lambda x: 1 if x == 2 else 0)
desocupados_totales = respondieron.groupby("anio")["desocupado"].sum()
print("\nCantidad de desocupados por año:")
print(desocupados_totales)

# Proporción de desocupados por nivel educativo
educacion_desocup = respondieron.groupby(["anio", "NIVEL_ED"])["desocupado"].mean().unstack()
educacion_desocup.T.plot(kind="bar")
plt.title("Proporción de desocupados por nivel educativo")
plt.ylabel("Proporción")
plt.xlabel("Nivel educativo")
plt.legend(title="Año")
plt.show()

# Proporción de desocupados por grupos etarios
def agrupar_edad(edad):
    if edad < 10:
        return "0-9"
    elif edad < 20:
        return "10-19"
    elif edad < 30:
        return "20-29"
    elif edad < 40:
        return "30-39"
    elif edad < 50:
        return "40-49"
    elif edad < 60:
        return "50-59"
    elif edad < 70:
        return "60-69"
    else:
        return "70+"

respondieron["grupo_edad"] = respondieron["CH06"].apply(agrupar_edad)
edad_desocup = respondieron.groupby(["anio", "grupo_edad"])["desocupado"].mean().unstack()
edad_desocup.T.plot(kind="bar", figsize=(10, 6))
plt.title("Proporción de desocupados por grupo de edad")
plt.ylabel("Proporción")
plt.xlabel("Grupo de edad")
plt.legend(title="Año")
plt.show()

# Diferencias por variable CH07 (vínculo familiar)
vinculo_desocup = respondieron.groupby(["anio", "CH07"])["desocupado"].mean().unstack()
vinculo_desocup.T.plot(kind="bar")
plt.title("Proporción de desocupados por vínculo familiar")
plt.ylabel("Proporción")
plt.xlabel("Vínculo familiar (CH07)")
plt.legend(title="Año")
plt.show()



FileNotFoundError: [Errno 2] No such file or directory: 'usu_individual_T124.xls'